In [4]:
import torch
torch.__version__
torch.cuda.current_device()
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 2080 Ti'

Basics

In [5]:
x = torch.arange(12, dtype = torch.float32)
x.shape
X = x.reshape(3,4)
X

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.]])

In [6]:
x = torch.tensor([1,2,3]).reshape(3,1)
y = torch.tensor([4,5,6]).reshape(1,3)

x, y


(tensor([[1],
         [2],
         [3]]),
 tensor([[4, 5, 6]]))

In [7]:
import os

os.makedirs(os.path.join('..','data'), exist_ok = True)
data_file = os.path.join('..', 'data', 'house_tiny.csv')

with open(data_file, 'w') as f:
    f.write('NumRooms,Alley,Price\n')  # Column names
    f.write('NA,Pave,127500\n')  # Each row represents a data example
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

In [8]:
import pandas as pd

data = pd.read_csv(data_file)
print(data)

   NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106000
2       4.0   NaN  178100
3       NaN   NaN  140000


In [9]:
inputs, outputs = data.iloc[:, 0:2], data.iloc[:,2]

inputs = inputs.fillna(inputs.mean())

inputs = pd.get_dummies(inputs, dummy_na = True)

print(inputs)

   NumRooms  Alley_Pave  Alley_nan
0       3.0           1          0
1       2.0           0          1
2       4.0           0          1
3       3.0           0          1


C:\Users\iamwl\miniconda3\envs\myenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
X, y = torch.tensor(inputs.values), torch.tensor(outputs.values)

X, y

(tensor([[3., 1., 0.],
         [2., 0., 1.],
         [4., 0., 1.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([127500, 106000, 178100, 140000]))

Autograd

In [14]:
x = torch.arange(4.0)
x.requires_grad_(True)
x, x.grad

(tensor([0., 1., 2., 3.], requires_grad=True), None)

In [16]:
y = 2 * torch.dot(x, x)

y

tensor(28., grad_fn=<MulBackward0>)

In [18]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [19]:
x.grad.zero_()
y = x * x
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])